In [1]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.19.0-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.19.0-preview"
#r "nuget: Azure.Identity"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Azure.Identity;

var builder = Kernel.CreateBuilder();

// EntraID authentication options.
DefaultAzureCredentialOptions defaultAzureCredentialOptions = new DefaultAzureCredentialOptions 
{ 
    ExcludeSharedTokenCacheCredential = true, 
    ExcludeEnvironmentCredential = true, 
    ExcludeAzurePowerShellCredential = true, 
    ExcludeInteractiveBrowserCredential = true, 
    ExcludeVisualStudioCredential = true, 
    ExcludeManagedIdentityCredential = true, 
    ExcludeVisualStudioCodeCredential = true, 
    ExcludeAzureCliCredential = false // Only use az login credentials
};

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
AzureOpenAIPromptExecutionSettings pes = new AzureOpenAIPromptExecutionSettings { MaxTokens = 100, Temperature = 0.001, TopP = 1.0, FrequencyPenalty = 0.0, PresencePenalty = 0.0 };  

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, new DefaultAzureCredential(defaultAzureCredentialOptions));
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Azure.Identity, 1.12.1 Microsoft.SemanticKernel, 1.22.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.19.0-preview Microsoft.SemanticKernel.Plugins.Core, 1.20.0-alpha

In [2]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
// Create a planner instance
var planner = new HandlebarsPlanner();

In [3]:
using System.ComponentModel;

using System;
using System.Collections.Generic;
using System.IO;
using System.Net.Http;

// Define the CompanyRecord class to hold the company data.
public record CompanyRecord
{
    public int Id { get; set; }
    public int Rank { get; set; }
    public string Name { get; set; }
    public string Industry { get; set; }
    public string City { get; set; }
    public string State { get; set; }
    public string Zip { get; set; }
    public string Website { get; set; }
    public int Employees { get; set; }
    public decimal RevenueInMillions { get; set; }
    public decimal ProfitInMillions { get; set; }
    public decimal ValuationInMillions { get; set; }
    public string Ticker { get; set; }
    public string CEO { get; set; }
}

// CompanyData plugin
public class CompanyDataPlugin
{
    HttpClient _httpClient = new HttpClient();
    [KernelFunction("find_company")]
    [Description("Find a company name given a ticker symbol or description of the company.")]
    [return: Description("A unique ID to the company. If the ID returned is -1, that means the company info could not be found - in which caswe the user should be informed about it.")]
    public async Task<int> GetCompanyNameAsync(string CompanyText)
    {
        // Call the API to find/validate the company name.
        return await Task.Run(() =>
        {
            var url = "http://localhost:5232/company_data/find/" + CompanyText;
            var r = _httpClient.GetAsync(url).Result;   
            if (r.StatusCode != System.Net.HttpStatusCode.OK)
            {
                return -1;
            }
            return int.Parse(r.Content.ReadAsStringAsync().Result);
        });
    }

    [KernelFunction("get_company_data")]
    [Description("Given the unique ID of a company, returns key data such as rank, industry, ticker symbol, address, website, valuation, profit, revenue, number of employees and CEO.")]
    [return: Description("The details of the company")]
    public async Task<CompanyRecord> GetCompanyDataAsync(string Id)
    {
        // Call the API to get the details of the company based on the ID.
        return await Task.Run(() =>
        {
            var url = "http://localhost:5232/company_data/" + int.Parse(Id);
            var r = _httpClient.GetAsync(url).Result;
            var json = JsonDocument.Parse(r.Content.ReadAsStringAsync().Result);
            var root = json.RootElement;
            return new CompanyRecord
            {
                Id = root.GetProperty("id").GetInt32(),
                Rank = root.GetProperty("rank").GetInt32(),
                Name = root.GetProperty("name").GetString(),
                Industry = root.GetProperty("industry").GetString(),
                City = root.GetProperty("city").GetString(),
                State = root.GetProperty("state").GetString(),
                Zip = root.GetProperty("zip").GetString(),
                Website = root.GetProperty("website").GetString(),
                Employees = root.GetProperty("employees").GetInt32(),
                RevenueInMillions = root.GetProperty("revenueInMillions").GetDecimal(),
                ProfitInMillions = root.GetProperty("profitInMillions").GetDecimal(),
                ValuationInMillions = root.GetProperty("valuationInMillions").GetDecimal(),
                Ticker = root.GetProperty("ticker").GetString(),
                CEO = root.GetProperty("ceo").GetString()
            };
        });
    }
}


In [4]:
#pragma warning disable SKEXP0050

//Add the plugins to the kernel
using Microsoft.SemanticKernel.Plugins.Core;
kernel.ImportPluginFromType<CompanyDataPlugin>();
kernel.ImportPluginFromType<MathPlugin>();
kernel.ImportPluginFromType<TextPlugin>();

In [5]:
#pragma warning disable SKEXP0060

var ask = "How much is the valuation and profitability of Fedex?";

var plan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Plan:\n");
Console.WriteLine(plan);

Plan:

{{!-- Step 0: Set the company name --}}
{{set "companyName" "Fedex"}}

{{!-- Step 1: Find the company ID using the CompanyDataPlugin-find_company helper --}}
{{set "companyId" (CompanyDataPlugin-find_company CompanyText=companyName)}}

{{!-- Step 2: Check if the company ID is valid --}}
{{#if (equals companyId -1)}}
  {{json "Company not found."}}
{{else}}
  {{!-- Step 3: Get the company data using the CompanyDataPlugin-get_company_data helper --}}
  {{set "companyData" (CompanyDataPlugin-get_company_data Id=companyId)}}

  {{!-- Step 4: Extract ValuationInMillions and ProfitInMillions from companyData --}}
  {{set "valuation" companyData.ValuationInMillions}}
  {{set "profitability" companyData.ProfitInMillions}}

  {{!-- Step 5: Output the valuation and profitability --}}
  {{json (concat "Valuation: $" valuation " million, Profitability: $" profitability " million")}}
{{/if}}


In [6]:
#pragma warning disable SKEXP0060

var planResult = await plan.InvokeAsync(kernel, new KernelArguments(pes));

Console.WriteLine("Plan results:\n");
Console.WriteLine(Utils.WordWrap(planResult.ToString(), 100));

Plan results:

Valuation: $57431 million, Profitability: $3826 million



In [7]:
#pragma warning disable SKEXP0060

ask = String.Empty;
while (ask != "exit")
{
    ask = await InteractiveKernel.GetInputAsync("Please ask your question about the company data");
    if (ask == "exit") continue;
    plan = await planner.CreatePlanAsync(kernel, ask);
    planResult = await plan.InvokeAsync(kernel, new KernelArguments());
    Console.WriteLine(Utils.WordWrap(planResult.ToString(), 100));
}
